# Direct Marketing Prediction

In this notebook, we are going to explore the direct marketing dataset. First, we start off with Exploratory Data Analysis, followed by feature engineering steps, walk through imputation, apply Machine Learning models, and interpret the results. We look at individual model performance, hypoerparameter tuning and look at different evaluation metrics. Later, we understand cost-benefit analysis for our problem and finally deploy a few models via. Airflow.

This notebook is solely for Data, features and model exploration, and is not the final deployment code. For more, visit [Direct Marketing Prediction Homepage](https://github.com/AdiVarma27/DirectMarketingPrediction).

#### Table Of Contents:

[Problem Statement & Data Import](#1)    
    1.1. [Problem Formulation](#1.1)    
    1.2. [Dataset import & columns exploration](#1.2)


[Exploratory Data Analysis](#2)            
    2.1. [Feature Exploration:Two-sample T-tests & Chi-square Tests](#2.1)     
    2.2. [Correlations](#2.2)       


[Feature Engineering](#3)     
    3.1. [Feature Transformation](#3.1)                  
    3.2. [Base Models](#3.2)           
   
   
[Modeling](#4)    
    4.1. [Inference Models, Logistic (L1, L2 tuning), tuning and metrics](#4.1)                  
    4.2. [Ensemble Methods](#4.2)   

---
## 1). Problem Statement & Data Import <a class="anchor" id="1"></a>

In this section, we are going to understand the task at hand and explore possible set of questions we could answer with the dataset, by looking at the columns at hand.

Our dataset comes from [UCI Repository](https://archive.ics.uci.edu/ml/datasets/bank+marketing) and original source used in the following paper:

**[A data-driven approach to predict the success of bank telemarketing](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf)**

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

### 1.1). Problem Formulation<a class="anchor" id="1.1"></a>

* From the above paper, we see that the source contains features of customers, who are **targeted through Telemarketing calls, to sell long-term deposits**. The dataset also includes calls where the client calls customer care for some reason, and long-term deposits are upsold at end of conversation as well. Hence, the dataset also contains a **label**, which denotes success or no-success on selling long-term deposits.


* The dataset comes from a rea Portuguese retail bank, from May 2008 to June 2013, with 52,944 phone calls in total. As expected with real world datasets, the dataset is imbalanced (different number of successful and unsuccessful outcome of treatment effect). The author also includes other interesting external resources such as socio-economic factors (Unemployment variation) and so on. We could also look at other socia-economic metrics about Portugal for those years and see if they might influence any purchasing decision making process.


* Given we have previous campaign information (campaign, contact and outcome), we could potentially find clusters of customers (Clustered by Treatment-Response). Hence, with respect to modeling, we have clear **Supervised Learning** & **Unsupervised Learning** tasks at hand. Other than that, we have a ton of interesting questions to answer with the dataset through **Exploratory Analysis & Visualizations**. Lets start by importing the required libraries & dataset.

### 1.2). Dataset Import <a class="anchor" id="1.2"></a>

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_columns', None)

# importing main dataset into DataFrame
df = pd.read_csv('dataset/bank-additional/bank-additional-full.csv')

# 41k rows, 21 columns
df.shape

(41188, 21)

In [2]:
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Dataset Columns

##### Demographic features:
age: Age of customers (numberic)       
job: Job description (categorical)       
marital: Marital Status (categorical)       
education: Eduation level of customer (categorical)      

##### Previous behaviour:
default: has credit default ? (categorical)      
housing: has housing loan ? (categorical)        
loan: has loan ? (categorical)        

##### Previous Treatment:
contact: contact via. (categorical)         
month, day_of_week_, duration: time based features (categorical, categorical, numberic)     

##### Campaign Information:
campaign: Number of times customer was contacted in this campaign (numeric)      
pdays: Number of days since last contact of previous campaign (numeric)          
previous: Number of times customer was contacted before this campaign (numeric)      
poutcome: Outcome of previous marketing campaign (categorical)         

##### Socio-Economic features:
emp.var.rate: Employment variation rate (numeric)
cons.price.idx: Consumer price index (numeric)
cons.conf.idx: Consumer confidence index (numeric)
euribor3m: Euribor 3 mon rate (numeric)
nr.employed: Number of employees (numeric)

##### Label: y (successful purchase or not, binary outcome)

---
## 2). Exploratory Data Analysis<a class="anchor" id="2"></a>
<a class="anchor" id="2.1"></a>
<a class="anchor" id="2.2"></a>
In this section, we are going to conduct exploratory data analysis, understand distribution, correlations and analyze features across label through visualizations. A few questions which might be interesting to explore are as follows:

* Does age, job, marital status and education play a role in predicting if the customer will make the purchase ? Are they statistically differences ?

* Is previous behaviour indicative of purchasing activity ?

* Do previous campaigns show any carry over effect ? How are previous outcomes related to next outcome ?

* Do socia-economica factors play a major role ? Is there any difference across the complete time period ?

### The above questions have been answered in [EDA notebook](https://github.com/AdiVarma27/DirectMarketingPrediction/blob/master/EDA.ipynb).

In [3]:
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


---
## 3). Feature Engineering <a class="anchor" id="3"></a>

In this section, we are going to look at missing values imputation and developing new features. We also look at feature transformations to prep the dataset for different machine learning algorithms.


### 3.1). Feature Transformations <a class="anchor" id="3.1"></a>

First, we need to convert the label (y), from categorical variables ('yes' or 'no'), to boolean/ numeric (1/0) and rename it to 'purchased?'

In [4]:
df['purchased?'] = df['y'].apply(lambda x: 1 if x=='yes' else 0)
df.drop(columns='y', inplace=True)
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,purchased?
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


##### For Machine Learning algorithms such as Logistic Regression, Shallow Neural Nets, SVM, Nearest Neighbours, we need to convert non0numeric attributes into OneHotEncoding dataframes.


##### For Tree based algorithms such as Decision Trees, Random Forests, Bagging & Boosting classifiers, we do not need additional feature trsnformations.

First, let us start with getting dummy variables for categorical columns, and prep data (named df_encoded) for first set of Machine Learning algorithms.

In [5]:
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,purchased?
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [6]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
purchased?        0
dtype: int64

At first look, we see that there are no null values. From further exploration, we see that nulls have been coded as 'unknown' across all columns. Hence, let us convert them back to null.

In [7]:
df = df.replace('unknown', np.nan)
pd.DataFrame(df.isnull().sum()/len(df)).sort_values(0, ascending=False).head()

,0
default,0.208726
education,0.042027
housing,0.024036
loan,0.024036
job,0.008012


After encoding 'unknown' cells back to null, we see many missing values. Let us walk through columns iteratively and decide what to do with them.

We observe that in default column, almost 21 % of rows are unknown. We might choose to drop it based on the context. For now, let us keep it in and see how feature selection methods remove the column (if it is not predictive of the outcome).

In [8]:
# categorical variables
cat_var = df[['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 
              'day_of_week','poutcome']]

# get dummies method to return binary variables for categories across categorical columns
df_prepped_list1 = pd.get_dummies(df, cat_var.columns)

df_prepped_list1.head(2)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,purchased?,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


### 3.2). Base Models <a class="anchor" id="3.2"></a>

#### Classifiers List 1: Logistic Regression, Knn, SVM.

Let us get the data ready to go for Base models (without any feature transformation, feature selection, hyperparameter tuning & cross-validation).

In [9]:
df.head(2)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,purchased?
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [10]:
df.poutcome.value_counts()

nonexistent    35563
failure         4252
success         1373
Name: poutcome, dtype: int64

About 11 % of dataset has label 1 and remianing label 0. Let us sample all data-points from rows with label 1 and and same number of points from rows with label 0, to balance both classes.

In [11]:
df_prepped_list1['purchased?'].sum(), len(df_prepped_list1)

(4640, 41188)

In [12]:
# concat equal sized dataframes
df_prepped_list1_model = pd.concat([df_prepped_list1[df_prepped_list1['purchased?'] == 1],  
                                    df_prepped_list1[df_prepped_list1['purchased?'] == 0].sample(4640) ])

In [13]:
len(df_prepped_list1_model)

9280

Now, let us split the dataset into train-test set (no validation dataset), into equal sizes.

In [14]:
y = df_prepped_list1_model['purchased?']
X = df_prepped_list1_model[['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_basic.4y', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'default_no', 'default_yes', 'housing_no', 'housing_yes', 'loan_no',
       'loan_yes', 'contact_cellular', 'contact_telephone', 'month_apr',
       'month_aug', 'month_dec', 'month_jul', 'month_jun', 'month_mar',
       'month_may', 'month_nov', 'month_oct', 'month_sep', 'day_of_week_fri',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success']]

import warnings
warnings.filterwarnings('ignore') 

In [15]:
# train test split
from sklearn.model_selection import train_test_split

# 0.5 split between train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=10)

#### Classifiers List 1: Logistic Regression, Knn, SVM.

In [18]:
# three models from List
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
nn = KNeighborsClassifier()
svm = SVC()

models = [lr, nn, svm]

train_acc = []
test_acc = []

for model in models:
    model.fit(X_train, y_train)
    train_acc.append(accuracy_score(model.predict(X_train), y_train))
    test_acc.append(accuracy_score(model.predict(X_test), y_test))

df_models1 = pd.DataFrame()
df_models1['models'] = models
df_models1['training accuracy'] = train_acc
df_models1['testing accuracy'] = test_acc

df_models1

,models,training accuracy,testing accuracy
0,LogisticRegression(),0.851940,0.848922
1,KNeighborsClassifier(),0.889655,0.846767
2,SVC(),0.818750,0.830603


Note that the above classifiers perform decently well, and have not been optimized at all (No hyper-parameter tuning, no cross validation or feature selection). We could take a look at ROC, precision-recall curves. However, we take a deeper look into them in the next section.

#### Classifiers List 2: Decision Tree, Random Forest, Boosting.

Let us repeat the same process as above, and split train-test dataset equally.

In [22]:
# three models from List
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
adb = AdaBoostClassifier()

models = [dt, rf, adb]

train_acc = []
test_acc = []

for model in models:
    model.fit(X_train, y_train)
    train_acc.append(accuracy_score(model.predict(X_train), y_train))
    test_acc.append(accuracy_score(model.predict(X_test), y_test))

df_models2 = pd.DataFrame()
df_models2['models'] = models
df_models2['training accuracy'] = train_acc
df_models2['testing accuracy'] = test_acc

df_models2

,models,training accuracy,testing accuracy
0,DecisionTreeClassifier(),1.000000,0.815733
1,"(DecisionTreeClassifier(max_features='auto', r...",1.000000,0.878879
2,"(DecisionTreeClassifier(max_depth=1, random_st...",0.870259,0.862500


---
## 4). Modeling <a class="anchor" id="4"></a>

In this section, we are going to apply few machine learning algorithms to predict customer purchase? columns in the dataset. Given the use case, we are better off starting with **highly-interpretable models** for **Inference** and end up using an **Ensemble** model for production/ deployment phase.

Let us start off with **Logistic Regression**, develop new features, use feature selection techniques, see how **Lasso & Ridge Classifiers** perform along with some hyperparameter tuning & cross-validation. Later, we are going to plot ROC, precision-recall and re-tune model if necessary.

### 4.1). Inference Modeling <a class="anchor" id="4.1"></a>

Let us start off by building a base-model for **Logistic Regression**, and work our way into fine-tuning it further. Here is our basic base model with balacned dataset.